# Aim: - Initial exploration of the Covid-19 dataset

In [3]:
# Importing the packages needed
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib

In [5]:
# Data 
counties_2020=pd.read_csv("../covid-19-data/us-counties-2020.csv")
counties_2021=pd.read_csv("../covid-19-data/us-counties-2021.csv")

In [6]:
counties_2020.head(5)

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0.0
3,2020-01-24,Cook,Illinois,17031.0,1,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0.0


In [8]:
# Lets look at the states and their corresponding deaths
counties_2020_groupby=counties_2020.groupby('state', as_index=False)[['cases','deaths']].sum()
counties_2020_groupby['death_per_cases']=round(counties_2020_groupby['deaths']/counties_2020_groupby['cases'],2)*100
counties_2020_groupby

,state,cases,deaths,death_per_cases
0,Alabama,32236000,526388.0,0.02
1,Alaska,2854127,13147.0,0.00
2,Arizona,47147078,1048821.0,0.02
3,Arkansas,18406368,285868.0,0.02
4,California,174969176,3065113.0,0.02
5,Colorado,23442034,542870.0,0.02
6,Connecticut,17214231,1119863.0,0.07
7,Delaware,5154377,145664.0,0.03
8,District of Columbia,3644621,146395.0,0.04
9,Florida,138123133,2632238.0,0.02
